In [58]:
import json
from datetime import datetime, time

from pymongo.mongo_client import MongoClient
import bson

from datetime import datetime
import os

* A partir du fichier trips_.json, remplacez les espaces des clés par un underscore, puis mettez en base en changeant les chaîne de caractères qui concerne des grandeurs numériques par le type integer ou float et les champs de date ($date) doit être sous format datetime.datetime. Ainsi les champs préfixé '$number...' suivent  l'exemple ci-dessous(cherchez du côté de la librairie bson, il existe une fonction utilitaire, il faut aussi réduire la profondeur):
* From the file trips_.json, replace the spaces in the keys with underscores. Then insert the data into the database while converting string values that represent numerical quantities into either integer or float types. Fields containing dates ($date) must be converted to the datetime.datetime format. Also, fields prefixed with $number... should follow the example below (look into the bson library — there is a utility function for this). The depth of the document should also be reduced.

```json
{'tripduration': {'$numberInt': 379}}
```

en :

```json
{'tripduration': 379}
```

In [59]:

def clean_document(doc):
    if isinstance(doc, dict):
        # Handle special MongoDB extended JSON values
        if "$numberInt" in doc:
            return int(doc["$numberInt"]) #If the document is like: {"$numberInt": "42"}, this will return 42 as a Python int.
        elif "$numberDouble" in doc:
            return float(doc["$numberDouble"])
        elif "$numberLong" in doc:
            return int(doc["$numberLong"])
        elif "$date" in doc:
            date_val = doc["$date"]
            if isinstance(date_val, dict) and "$numberLong" in date_val:
                return datetime.fromtimestamp(int(date_val["$numberLong"]) / 1000.0) #Dates are stored in JSON as a Unix timestamp in milliseconds Python's datetime.fromtimestamp() expects seconds, so we divide by 1000.
            elif isinstance(date_val, str): #If the date is an ISO-formatted string
                return datetime.fromisoformat(date_val.replace("Z", "+00:00")) #.replace("Z", "+00:00") makes it compatible with datetime.fromisoformat
                                                                                #Converts the string to a native datetime object

        # Clean key names and recursively clean values
        cleaned = {}
        for key, value in doc.items():
            clean_key = key.replace(" ", "_")
            cleaned[clean_key] = clean_document(value) #handling normal dictionaries (not just special MongoDB fields)
        return cleaned

    elif isinstance(doc, list): #f doc is a list
        return [clean_document(item) for item in doc] #this goes through each item in the list and applies clean_document() recursively.
    
    else:
        return doc #If the input doc is: a string ("hello"), an integer (123), a float (3.14), None, or anything that isn’t a dict or list…, 
                   #Then it just returns it as-is.


In [60]:
import json

cleaned_documents = [] #Creates an empty list that will store all the cleaned documents.

with open("trips_.json", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip() #removes leading/trailing whitespace and newline characters
        if not line:
            continue
        raw_doc = json.loads(line) #converts the JSON string into a Python dictionary 
        cleaned = clean_document(raw_doc) #applies the cleaning function I defined earlier
        cleaned_documents.append(cleaned) #Adds the cleaned version of the document to the cleaned_documents list

# Show one example
print(cleaned_documents[0])



{'tripduration': 307, 'start_station_id': 3118, 'start_station_name': 'McGuinness Blvd & Eagle St', 'end_station_id': 3119, 'end_station_name': 'Vernon Blvd & 50 Ave', 'bikeid': 23477, 'usertype': 'Subscriber', 'birth_year': 1987, 'gender': 1, 'start_station_location': {'type': 'Point', 'coordinates': [-73.95284, 40.73555]}, 'end_station_location': {'type': 'Point', 'coordinates': [-73.95411749, 40.74232744]}, 'start_time': datetime.datetime(2016, 1, 1, 8, 4, 31), 'stop_time': datetime.datetime(2016, 1, 1, 8, 9, 38)}


Après avoir mis en base et uniquement, assurez-vous qu'il n'y a pas de doublons, si il y en a, écrivez une fonction python permettant de les retirer (sans à avoir à faire de téléchargement) 'delete dublicates'

In [ ]:
# Connect to local MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["bikedata"]
collection = db["trips"]

# Insert documents
collection.insert_many(cleaned_documents)
print(f"Inserted {len(cleaned_documents)} documents.")


Inserted 10019 documents.


In [62]:
from bson.json_util import dumps #converts a Python dict (or MongoDB document) into a JSON-formatted string, 
                                #including support for special MongoDB types like ObjectId, datetime, etc.

def remove_duplicates(collection):
    seen = set()
    duplicates = []

    for doc in collection.find(): #Loop Through Every Document in the Collection
        doc_copy = doc.copy() #Prepare the Document for Comparison
        doc_copy.pop("_id", None) #Even if two documents are otherwise identical, their _ids will differ, so we remove _id before comparison.
        doc_str = dumps(doc_copy, sort_keys=True) #Converts the document into a string using bson.json_util.dumps()
                                                #sort_keys=True ensures a consistent order for fields 

        if doc_str in seen: #If this stringified document has been seen before → it’s a duplicate → store its _id in duplicates
            duplicates.append(doc["_id"])
        else:
            seen.add(doc_str) #If not → add it to seen

    if duplicates:
        collection.delete_many({"_id": {"$in": duplicates}}) #Uses $in to match any document with one of those _ids
        print(f"Removed {len(duplicates)} duplicates.")
    else:
        print("No duplicates found.")

remove_duplicates(collection)


Removed 10019 duplicates.


In [63]:
output_path = os.path.join(os.getcwd(), "cleaned_trips.json")

with open(output_path, "w", encoding="utf-8") as f:
    for doc in cleaned_documents:
        json.dump(doc, f, default=str)  # default=str for datetime
        f.write("\n")

print("Saved cleaned_trips.json to:")
print(output_path)

Saved cleaned_trips.json to:
/Users/pernebayarailym/Documents/Portfolio_Projects_AP/Simplon_DE_Projects/Python_Projects/MongoDB_practice/cleaned_trips.json


De plus, des hypothèses d'incohérences ont été émises par les parties prenantes, il s'agit d'écrire pour chacune de ces hypothèses une fonction python qui va détecter les documents incriminés et les flagger avec un champs supplémentaire pour rectification/enquête ultérieure :

* Un vélo loué deux fois, mais la deuxième période de location démarre avant le rendu de la première période de location, ce qui n'est pas normal, ou tout autre chevanchement de période.
* Un utilisateur trop jeune (le service est réservé au + de 13 ans)
* La date de naissance n'est pas renseignée
* Des locations trop courtes (1 secondes)
* Des temps de location incohérents par rapport au start_time et au end_time


Faites un rapport détaillé des anomalies trouvées (nombres d'occurences)

* In addition, stakeholders have raised hypotheses about potential data inconsistencies. You are required to write a Python function for each of these hypotheses to detect the problematic documents and flag them with an additional field for future correction/investigation:

- A bike is rented twice, but the second rental period starts before the first one has ended, which is abnormal, or any other overlapping rental periods.
- A user who is too young (the service is restricted to those over 13 years old).
- The birth date is missing.
- Very short rentals (e.g., 1 second long).
- Inconsistent rental durations compared to the recorded start_time and end_time.

Create a detailed report of the anomalies found (number of occurrences).

Puis, après avoir fait un nettoyage et écarté les locations suspectes, répondez aux questions métier qui suivent :

* Quels sont les 5 trajets (start station → end station) les plus fréquents pour les utilisateurs de genre féminin ? (gender = 1)

* Quel est le nombre total de trajets par type d’utilisateur (Subscriber vs Customer) pour l’année 2023 ?

* Quelle est la durée moyenne des trajets par station de départ pour les trajets commençant entre 7h et 9h ?

* Quel est le top 3 des stations avec la plus forte augmentation du nombre de trajets entre deux années consécutives ?

* Pour chaque catégorie d’âge, quelle est la durée médiane des trajets ?

* Quels sont les utilisateurs les plus actifs (10%) et leur contribution au nombre total de trajets ?

* Quelle est la répartition des trajets par jour de la semaine (lundi, mardi, etc.) ?

* Quel est le temps moyen passé en trajet pour chaque genre, filtré sur les trajets de plus de 10 minutes ?

* Combien de trajets ont démarré dans une station donnée pendant les heures de pointe (ex. 7h-9h et 17h-19h) ?

* Quelles sont les 5 stations avec la plus grande diversité d’utilisateurs (nombre distinct d’user id) ?